In [2]:
import pandas as pd, numpy as np, calendar as cal, datetime as dt

In [3]:
df = pd.read_csv("DATA/data-social-network/edges.csv",sep =',',header = None)

In [4]:
df.columns = ['FROM', 'TO']

In [5]:
df.head()

,FROM,TO
0,151838,317823
1,151838,91631
2,151838,91943
3,151838,92060
4,151838,92073


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14067887 entries, 0 to 14067886
Data columns (total 2 columns):
FROM    int64
TO      int64
dtypes: int64(2)
memory usage: 214.7 MB


In [7]:
data = pd.read_csv("DATA/data-social-network/nodes.csv",sep =',',header = None)

In [8]:
data.columns = ['NODE']

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100199 entries, 0 to 100198
Data columns (total 1 columns):
NODE    100199 non-null int64
dtypes: int64(1)
memory usage: 782.9 KB


In [32]:
df_unique_from= pd.DataFrame(df.FROM.unique())
#df_unique_from.info()
df_unique_from.columns = ['NODE']
df_unique_from.head()

,NODE
0,151838
1,100000
2,10000
3,100001
4,100002


In [30]:
df_unique_to=pd.DataFrame( df.TO.unique())
#df_unique_to.columns('NODE')
df_unique_to.columns = ['NODE']
df_unique_to.head()

,NODE
0,317823
1,91631
2,91943
3,92060
4,92073


In [31]:
df_unique_to.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5689497 entries, 0 to 5689496
Data columns (total 1 columns):
NODE    int64
dtypes: int64(1)
memory usage: 43.4 MB


In [33]:
df_concat_from_to = pd.concat([df_unique_to, df_unique_from])

In [34]:
df_concat_from_to.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5826829 entries, 0 to 137331
Data columns (total 1 columns):
NODE    int64
dtypes: int64(1)
memory usage: 88.9 MB


In [35]:
df_node_total= pd.concat([df_concat_from_to, data])


In [36]:
df_node_total.head()

,NODE
0,317823
1,91631
2,91943
3,92060
4,92073


In [ ]:
151838

In [53]:
df_unique_node_total=pd.DataFrame( df_node_total.NODE.unique())

In [54]:
df_unique_node_total.columns = ['NODE']

In [55]:
df_unique_node_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5689498 entries, 0 to 5689497
Data columns (total 1 columns):
NODE    int64
dtypes: int64(1)
memory usage: 43.4 MB


In [39]:
from arango import ArangoClient

In [40]:
# Initialize the client for ArangoDB.
client = ArangoClient(protocol='http', host='localhost', port=8529)

In [47]:
# Connect to "test" database as root user.
db = client.db('NFE204', username='root', password='password')

In [48]:

# Create a new graph named "friendster".
graph = db.create_graph('friendster')

In [49]:
# Create vertex collections for the graph.
nodes = graph.create_vertex_collection('nodes')


In [56]:
#insertion des nodes dans la collection nodes
for item, row in df_unique_node_total.iterrows():
    nodes.insert({'_key': str(row['NODE']),'name': int(row['NODE'])})



ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [57]:
register = graph.create_edge_definition(
    edge_collection='edges',
    from_vertex_collections=['nodes'],
    to_vertex_collections=['nodes']
)


In [58]:
register

<EdgeCollection edges>

In [69]:
df_error=pd.DataFrame(np.array([0]))

In [71]:
df_error.columns = ['MSG']

In [72]:
df_error

,MSG
0,0


In [ ]:
#insertions des edges dans la collection edges
for item, row in df.iterrows():
    fromstr = 'nodes/'+str(row['FROM'])
    tostr   = 'nodes/'+str(row['TO'])
    # Insert edge documents into "register" edge collection.
    try:
        register.insert({'_from': fromstr, '_to': tostr})
    except:      
        df_error.append(pd.Series([int(row['TO']),int(row['FROM'])]),ignore_index=True)


In [26]:
# Traverse the graph in outbound direction, breadth-first.
result = graph.traverse(
    start_vertex='nodes/28732892',
    direction='outbound',
    strategy='breadthfirst'
)

In [ ]:
df_error.head()

In [27]:
result

{'paths': [{'edges': [],
   'vertices': [{'_id': 'nodes/28732892',
     '_key': '28732892',
     '_rev': '_XB1ilNy--_',
     'name': 13529}]}],
 'vertices': [{'_id': 'nodes/28732892',
   '_key': '28732892',
   '_rev': '_XB1ilNy--_',
   'name': 13529}]}